In [ ]:
import numpy as np 
import math
from scipy.stats import bernoulli
from scipy.stats import beta
import matplotlib.pyplot as plt 
from scipy.optimize import minimize

In [ ]:
np.log(2)

In [ ]:
np.arange(1,10)**2

In [ ]:
def wealth_process(seq, alpha=0.05): 
    # Construct wealth process from sequence X using ONS 
    
    wealth = 1
    wealth_hist = [1] 
    const = 2 / (2 - np.log(3))
    lambd = 0 
    zt2 = 0 
    for t in range(len(seq)): 
                
        St = 1 - lambd*seq[t]
        wealth = wealth * St 
        wealth_hist.append(wealth)
        if wealth > 2/alpha: 
            #print(f"Reject at time {t}")
            break 
        
        # Update lambda via ONS  
        z = seq[t] / (1 - lambd*seq[t])
        zt2 += z**2
        lambd = max(min(lambd - const*z/(1 + zt2), 1/2), -1/2)
        
    return wealth_hist
           

In [ ]:
import matplotlib as mpl 


In [ ]:
mpl.rcParams['font.size'] = 14

In [ ]:
size = 300
X1 = bernoulli.rvs(0.7, size=size)
X2 = bernoulli.rvs(0.4, size=size)
X3 = bernoulli.rvs(0.4, size=size)
X4 = bernoulli.rvs(0.55, size=size)
seq = X1 - X2
wealth = wealth_process(seq, alpha=0.01)
plt.plot(range(len(wealth)), wealth, label='$\Delta=0.3$', c='navy')
seq = X3 - X4
wealth = wealth_process(seq, alpha=0.01)
plt.plot(range(len(wealth)), wealth, label='$\Delta = 0.1$', c='purple')
seq = X2 - X3
wealth = wealth_process(seq, alpha=0.01)
plt.plot(range(len(wealth)), wealth, label='$\Delta=0$', c='olive')

plt.title('Growth of wealth process', fontsize=14)
plt.yscale('log')
plt.ylabel('Wealth')
plt.xlabel('Time')
plt.legend()

plt.savefig('plots/wealth_proc.png', dpi=300, bbox_inches='tight')

In [ ]:
deltas = np.linspace(0, 1, 40)

 
times = []
n_iters = 50 
for _ in range(n_iters): 
    ts = []
    for delta in deltas: 
        X1 = bernoulli.rvs(0.5 - delta/2, size=500)
        X2 = bernoulli.rvs(0.5 + delta/2, size=500)
        seq = X1 - X2
        wealth = wealth_process(seq, alpha=0.01)
        ts.append(len(wealth))
    times.append(ts)
    


In [ ]:
avg_times = np.mean(times, axis=0)[3:]
std = np.std(times, axis=0)[3:]

plt.plot(deltas[3:], avg_times, lw=3)
plt.fill_between(deltas[3:], avg_times + std, avg_times - std, alpha=0.2)
plt.title('Time to rejection under alternative', fontsize=14)
plt.xlabel('$|\mu_0-\mu_1|$')
plt.ylabel('Time')
plt.yscale('log')

plt.savefig('plots/time_to_reject.png', dpi=300, bbox_inches='tight')